In [1]:
import os
import pandas as pd
from pathlib import Path

Data taxis verdes

Esta funcion sirve para realizar la carga de datos

In [3]:
def load_parquet_files(folder_path, Año):
    parquet_files = Path(folder_path).glob('*.parquet')#Obtenemos una lista de rutas a archivos .parquet en la carpeta especificada
    parquet_files_Año = [file for file in parquet_files if f'green_tripdata_{Año}' in file.name]#filtramos el nombre del archivo con el que vamos trabajar

    data_frames_Año = []#iniciamos un lista para almacenar los DataFrames cargados desde los archivos .parquet
    for file in parquet_files_Año:# Iteramos a través de los archivos .parquet correspondientes al año y cargar cada uno en un DataFrame
        df = pd.read_parquet(file)# Cargamos el archivo parquet
        data_frames_Año.append(df)# Agregamos el DataFrame cargado a la lista

    return data_frames_Año # Devolvemos la lista de DataFrames correspondientes al año


Función para transformar datos

In [4]:
def transform_data(data_frames, Año):
    combined_df = pd.concat(data_frames, ignore_index=True)# Concatenamos los DataFrames de la lista en uno solo
      # Convertimos columnas de fecha y hora de recogida y entrega a formato datetime
    combined_df['lpep_pickup_datetime'] = pd.to_datetime(combined_df['lpep_pickup_datetime'])
    combined_df['lpep_dropoff_datetime'] = pd.to_datetime(combined_df['lpep_dropoff_datetime'])
    combined_df['pickup_date'] = combined_df['lpep_pickup_datetime'].dt.date# Creamos una nueva columna "pickup_date" con la fecha de recogida (sin la hora)
        # Cambiamos las columna a formato datetime
    df_taxi = combined_df.copy()
    df_taxi['pickup_date'] = pd.to_datetime(df_taxi['pickup_date'])
    df_taxi['lpep_pickup_datetime'] = pd.to_datetime(df_taxi['lpep_pickup_datetime'])
    df_taxi['lpep_dropoff_datetime'] = pd.to_datetime(df_taxi['lpep_dropoff_datetime'])
        # Ordemanos las columnas
    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxi.columns if col not in ['VendorID', 'pickup_date']]
    df_taxi = df_taxi[column_order]
     # Filtramos el DataFrame para incluir solo registros del año especificado
    df_taxi = df_taxi[df_taxi['pickup_date'].dt.year == Año]
    #Eliminamos las columnas que no son reelevantes para nuestro DataFrame
    columns_to_remove = ['extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxi = df_taxi.drop(columns=columns_to_remove)

    return df_taxi #Nos devuelve el Dataframe ya listo para guardarse

Realizamos los llamados a las funciones 

In [5]:

folder_path = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo'
Año = 2021
data_frames_Año = load_parquet_files(folder_path, Año)
df_taxiG= transform_data(data_frames_Año, Año)
df_taxiG.head(1)



,VendorID,pickup_date,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,total_amount,payment_type,trip_type
0,2,2021-01-01,2021-01-01 00:15:56,2021-01-01 00:19:52,1.0,43,151,1.0,1.01,5.5,6.8,2.0,1.0


Guardamos el DataFrame con el nombre correspondiente al año elegido

In [63]:
df_taxiG.to_csv(f'taxiV_{Año}.csv') 

Data Taxis amarillos

Se realiza el mismo procedimiento que en la funcion anterior, con la diferencia que al ser archivos mas pesados se deben crear DataFrame por mes 

Función para la carga de datos

In [34]:
def load_parquet_filesY(folder_path, AñoY):
    parquet_files = Path(folder_path_Y).glob('*.parquet')
    parquet_files_Año = [file for file in parquet_files if f'yellow_tripdata_{AñoY}-{mes}' in file.name]

    data_frames_AñoY = []
    for file in parquet_files_Año:
        df = pd.read_parquet(file)
        data_frames_AñoY.append(df)

    return data_frames_AñoY


Funcion de Transformación de datos

In [5]:
def transform_data(data_frames, AñoY):
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['tpep_pickup_datetime'] = pd.to_datetime(combined_df['tpep_pickup_datetime'])
    combined_df['tpep_dropoff_datetime'] = pd.to_datetime(combined_df['tpep_dropoff_datetime'])
    combined_df['pickup_date'] = combined_df['tpep_pickup_datetime'].dt.date

    df_taxiY = combined_df.copy()
    df_taxiY['pickup_date'] = pd.to_datetime(df_taxiY['pickup_date'])
    df_taxiY['tpep_pickup_datetime'] = pd.to_datetime(df_taxiY['tpep_pickup_datetime'])
    df_taxiY['tpep_dropoff_datetime'] = pd.to_datetime(df_taxiY['tpep_dropoff_datetime'])

    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxiY.columns if col not in ['VendorID', 'pickup_date']]
    df_taxiY = df_taxiY[column_order]

    df_taxiY = df_taxiY[df_taxiY['pickup_date'].dt.year == AñoY]

    columns_to_remove = ['extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxiY = df_taxiY.drop(columns=columns_to_remove)

    return df_taxiY

Llamado a las funciones y guardado de los DataFrame

In [36]:
folder_path_Y = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo\\Amarillo'
AñoY = 2022
mes=12
data_frames = load_parquet_filesY(folder_path_Y, AñoY)
df_taxiY = transform_data(data_frames, AñoY)
df_taxiY.info()
df_taxiY.to_csv(f'taxiY_{AñoY}_{mes}.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 3252711 entries, 0 to 3252716
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   pickup_date            datetime64[ns]
 2   tpep_pickup_datetime   datetime64[ns]
 3   tpep_dropoff_datetime  datetime64[ns]
 4   passenger_count        float64       
 5   trip_distance          float64       
 6   RatecodeID             float64       
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  total_amount           float64       
 12  airport_fee            float64       
dtypes: datetime64[ns](3), float64(6), int64(4)
memory usage: 347.4 MB


Combinacion de DataFrame

Función de carga de datos

In [11]:
def load_csv_files(folder_path, year):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and f'taxiY_{year}' in file]
    
    data_frames = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    
    return data_frames

Función de transformación de DF

In [9]:
def combine_and_transform_data(data_frames):
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

In [17]:
folder_path = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo'
year = 2022

data_frames = load_csv_files(folder_path, year)
combined_data = combine_and_transform_data(data_frames)

In [23]:
combined_data.to_csv('taxiY_{year}.csv')

In [2]:
df=pd.read_csv('taxiY_2022.csv')

: 

: 

In [ ]:
import numpy as np
import datetime as dt 

In [ ]:
dfElectric=pd.read_csv('Electric and Alternative Fuel Charging Stations.csv')

In [ ]:
dfElectric.info()

In [ ]:
dfElectric.head()

In [ ]:
estaciones_ny = dfElectric[dfElectric['State'] == 'NY']

In [ ]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['Street Address','Intersection Directions','ZIP','Plus4','Station Phone','Status Code','Groups With Access Code',
'Access Days Time','Cards Accepted','Date Last Confirmed','Updated At','Owner Type Code','Federal Agency ID',
'Open Date','Country','Access Code','Facility Type','CNG On-Site Renewable Source','CNG Total Compression Capacity','CNG Storage Capacity','EV Pricing',
'LPG Nozzle Types','CNG Fill Type Code','CNG PSI','EV On-Site Renewable Source','Restricted Access','Expected Date','BD Blends','NG Fill Type Code','NG PSI',
'EV Other Info','EV Network Web','Hydrogen Status Link','LPG Primary', 'E85 Blender Pump', 'Intersection Directions (French)','Access Days Time (French)','BD Blends (French)',
'Hydrogen Is Retail','Federal Agency Code','LNG On-Site Renewable Source','E85 Other Ethanol Blends','EV Pricing (French)','Hydrogen Pressures','Hydrogen Standards','Federal Agency Name'
]

# Elimina las columnas especificadas
estaciones_ny = estaciones_ny.drop(columns=columnas_a_eliminar)

In [ ]:
# Reorganiza las columnas
column_order = ['ID'] + [col for col in estaciones_ny.columns if col != 'ID']
estaciones_ny=estaciones_ny[column_order]

In [ ]:
estaciones_ny.info()

In [ ]:
estaciones_ny.head(3)

In [ ]:
estaciones_ny.to_csv('Station_NY.csv',index=False)

In [ ]:
dftaxi_zone=pd.read_csv('Dataset_extraidos\\taxi_zones.csv')

In [ ]:
dftaxi_zone.head(5)

In [ ]:
dftaxi_zone.info()

In [ ]:
columns_to_drop = ['Unnamed: 0', 'OBJECTID']
dftaxi_zone = dftaxi_zone.drop(columns=columns_to_drop)

In [ ]:
column_order = ['LocationID'] + [col for col in dftaxi_zone.columns if col != 'LocationID']
dftaxi_zone=dftaxi_zone[column_order]

In [ ]:
dftaxi_zone = dftaxi_zone.rename(columns={'y': 'longitude', 'x': 'latitude'})

In [ ]:
dftaxi_zone.to_csv('Taxi zone.csv',index=False)